In [ ]:
import h5py
import wfdb
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt  # Added for filtering
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models, utils, callbacks
from tqdm import tqdm
import tensorflow as tf
import os
import matplotlib.pyplot as plt

In [ ]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(r_signal, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order = order)
    f_signal = filtfilt(b, a, r_signal)
    return f_signal

In [ ]:
record_dir = "F:\\physionet.org\\files\\music-sudden-cardiac-death\\1.0.1\\Holter_ECG"
record_path = f"{record_dir}/P0043"
record = wfdb.rdrecord(record_path, channels=[0, 1, 2]) # Read only the first 3 leads by index
signals = record.p_signal.astype(np.float32)
fs = record.fs
lead_1 = signals[:, 0]
lead_2 = signals[:, 1]
lead_3 = signals[:, 2]

In [ ]:
r_signal = lead_1
f_signal = butter_bandpass_filter(r_signal, 1, 40, fs, order=5)

plt.figure(figsize=(20,3))
plt.plot(r_signal)

plt.plot(f_signal,'r')
plt.show()

In [ ]:
plt.figure(figsize=(20,2))
plt.plot(r_signal[15000:26000])
plt.plot(f_signal[15000:26000])

In [1]:
import os
import logging
import datetime
import re
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
import wfdb
from scipy.signal import butter, lfilter, filtfilt
from IPython import get_ipython
import matplotlib.pyplot as plt

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(r_signal, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    f_signal = filtfilt(b, a, r_signal)
    return f_signal

import os
import numpy as np
import matplotlib.pyplot as plt

def plot_and_save_single_ecg(signal, patient_id, output_dir, sampling_rate, plot_filename):
    """
    Plots a single ECG signal and saves it as a separate image file.

    Args:
        signal (np.array): A 1D numpy array representing the ECG signal.
        patient_id (str or int): The patient ID corresponding to the signal.
        output_dir (str): The directory where the image files will be saved.
        sampling_rate (float): The sampling rate of the ECG signal (samples per second).
        plot_filename (str): The specific filename for this plot.
    """
    os.makedirs(output_dir, exist_ok=True)

    time = np.arange(signal.shape[0]) / sampling_rate

    fig, ax = plt.subplots(figsize=(10, 5)) # Adjust figsize as needed for a single plot
    ax.plot(time, signal)
    ax.grid(False) # Removes the grid lines

    # Remove tick labels from x and y axes
    ax.set_xticks([])
    ax.set_yticks([])

    # You had these commented out in your previous response. If you want a title, uncomment:
    # ax.set_title(f"Patient ID: {patient_id}", fontsize=12)
    # ax.set_xlabel("Time (seconds)", fontsize=10)
    # ax.set_ylabel("Amplitude", fontsize=10)
    # ax.tick_params(axis='both', which='major', labelsize=8) # This line controls label size, but set_xticks/yticks will make them disappear.

    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, plot_filename))
    plt.close(fig) # Close the figure to free up memory


def process_and_inspect_ecg(patient_ids, record_dir, refTable, inspection_dir, sampling_rate, window_seconds=5, start_offset_seconds=3600, lead_index=0, inspection_frequency=1):
    """
    Processes ECG signals and periodically saves individual ECG windows for inspection.

    Args:
        patient_ids (list): List of patient IDs to process.
        record_dir (str): Directory containing the WFDB record files.
        refTable (pd.DataFrame): DataFrame containing patient information.
        inspection_dir (str): Directory to save inspection plot images.
        sampling_rate (int): Expected sampling rate of the ECG signals.
        window_seconds (int): The size of the window in seconds to extract.
        start_offset_seconds (int): Starting offset in seconds.
        lead_index (int): The index of the lead to process.
        inspection_frequency (int): Save an inspection plot every N processed patients.
    """
    os.makedirs(inspection_dir, exist_ok=True)

    target_window_size = int(window_seconds * sampling_rate)
    start_index = int(start_offset_seconds * sampling_rate)

    for i, pid in enumerate(tqdm(patient_ids, desc=f"Processing ECGs for inspection ({window_seconds} sec, Lead {lead_index})")):
        record_path = f"{record_dir}/{pid}"
        try:
            record = wfdb.rdrecord(record_path, channels=[lead_index])
            signals = record.p_signal.astype(np.float32)
            fields = record.__dict__
            fs = fields['fs']

            if fs != sampling_rate:
                print(f"Sampling rate mismatch for patient {pid}. Skipping.")
                continue
            if signals.shape[1] < 1:
                print(f"Less than 1 lead for patient {pid}. Skipping.")
                continue
            if len(signals) < start_index + target_window_size:
                print(f"Signal too short for patient {pid}. Skipping.")
                continue

            window = signals[start_index:start_index + target_window_size, 0]
            filtered_signal = butter_bandpass_filter(window, 1, 40, fs, order=5)

            patient_row = refTable[refTable['Patient_ID'] == pid]
            if patient_row.empty:
                print(f"Patient {pid} not in refTable. Skipping.")
                continue

            # --- ECG plot generation for inspection ---
            if (i + 1) % inspection_frequency == 0:
                plot_filename = f"{pid}.png"
                plot_and_save_single_ecg(filtered_signal, pid, inspection_dir, sampling_rate, plot_filename)
            # --- End of ECG plot generation ---

        except Exception as e:
            print(f"Error processing patient {pid}: {e}")

In [3]:
import pandas as pd
import os
# Assuming setup_logger is no longer needed/defined if skipping logging

# Example usage
if __name__ == '__main__':
    data_prep_dir = "./data_preparation"
    os.makedirs(data_prep_dir, exist_ok=True)
    # log_directory = os.path.join(data_prep_dir, "logs") # No longer needed
    inspection_directory = os.path.join(data_prep_dir, "ecg-images")

    # logger = setup_logger(log_directory) # No longer needed
    # logger.info("Starting TFRecord generation with inspection (9 plots per image).") # No longer needed

    refTable = pd.read_csv('resources/reference-table.csv')
    all_patients = refTable['Patient_ID'].tolist()
    num_patients = len(all_patients)

    record_directory = "F:\\physionet.org\\files\\music-sudden-cardiac-death\\1.0.1\\Holter_ECG"
    # output_directory = os.path.join(data_prep_dir, "tfrecords-5seconds-singlelead") # No longer needed
    reference_table = refTable
    sampling_rate = 200
    train_patient_ids = all_patients[:num_patients]
    lead_to_process = 1 # Index of lead
    window_size_seconds = 5
    inspection_frequency = 1
    # plots_per_image = 9 # No longer needed, as it's one plot per image

    # Call the updated function
    process_and_inspect_ecg(
        train_patient_ids,
        record_directory,
        reference_table,
        inspection_directory,
        sampling_rate,
        window_size_seconds,
        lead_index=lead_to_process,
        inspection_frequency=inspection_frequency
    )

    print("ECG inspection plot generation completed.") # Updated success message

Processing ECGs for inspection (5 sec, Lead 1):  26%|██▌       | 177/694 [02:56<09:08,  1.06s/it]

Error processing patient P0300: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P0300.hea'


Processing ECGs for inspection (5 sec, Lead 1):  32%|███▏      | 220/694 [03:39<09:22,  1.19s/it]

Error processing patient P0368: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P0368.hea'
Error processing patient P0369: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P0369.hea'
Error processing patient P0370: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P0370.hea'


Processing ECGs for inspection (5 sec, Lead 1):  34%|███▎      | 233/694 [03:50<07:31,  1.02it/s]

...

Error processing patient P1028: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P1028.hea'


Processing ECGs for inspection (5 sec, Lead 1):  99%|█████████▊| 684/694 [11:25<00:07,  1.30it/s]

Error processing patient P1033: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P1033.hea'


Processing ECGs for inspection (5 sec, Lead 1): 100%|██████████| 694/694 [11:33<00:00,  1.00it/s]

Error processing patient P1083: [Errno 2] No such file or directory: 'F:/physionet.org/files/music-sudden-cardiac-death/1.0.1/Holter_ECG/P1083.hea'
ECG inspection plot generation completed.
